<a href="https://colab.research.google.com/github/SytzeAndr/NGCF_RP32/blob/master/NGCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import random as rd
from google.colab import drive

**File loading**

Here we use the Google Drive mountpoint to load files. For this to work, note the following:



*   The first time you execute this, it will provide a link, which you need to follow and give permission for Colab to access your Google Drive.
*   Make sure that the data is located in the folder `RP_data` which should be located in the root of your Drive.

In [0]:
drive.mount('/content/drive')
data_path = './drive/My Drive/RP_data'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**DataReader class**

The DataReader class is a utility class that will help loading the data, compute useful properties of the data and create sample batches.

In [0]:
class DataReader(object):
  def __init__(self, path='./drive/My Drive/RP_data', batch_size=100):
    self.path = path
    self.batch_size = batch_size

    train_file = path + '/train.txt'
    test_file = path + '/test.txt'

    self.n_train = 0
    self.n_test = 0
    self.exist_users = []
    self.train_items = {}
    self.test_items = {}

    with open(train_file) as f:
      for l in f.readlines():
        if len(l) > 0:
          l = l.strip('\n')
          items = [int(i) for i in l.split(' ')]
          uid, train_items = items[0], items[1:]
          self.exist_users.append(uid)
          self.train_items[uid] = train_items
          self.n_train += len(train_items)

    with open(test_file) as f:
      for l in f.readlines():
        if len(l) > 0:
          l = l.strip('\n')
          try:
            items = [int(i) for i in l.split(' ')]
            uid, test_items = items[0], items[1:]
            self.exist_users.append(uid)
            self.test_items[uid] = test_items
            self.n_test += len(test_items)
          except Exception:
            continue

    train_max_item = max([max(items) for items in list(self.train_items.values())])
    test_max_item = max([max(items) for items in list(self.test_items.values())])
    self.n_items = max(train_max_item, test_max_item)
    self.n_users = max(self.exist_users)
  

  def sample(self):
    if self.batch_size <= self.n_users:
      users = rd.sample(self.exist_users, self.batch_size)
    else:
      users = [rd.choice(self.exist_users) for _ in range(self.batch_size)]


    def sample_pos_items_for_u(u, num):
      pos_items = self.train_items[u]
      n_pos_items = len(pos_items)
      pos_batch = []
      while True:
        if len(pos_batch) == num: break
        pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
        pos_i_id = pos_items[pos_id]

        if pos_i_id not in pos_batch:
          pos_batch.append(pos_i_id)
      return pos_batch


    def sample_neg_items_for_u(u, num):
      neg_batch = []
      while True:
        if len(neg_batch) == num: break
        neg_id = np.random.randint(low=0, high=self.n_items, size=1)[0]
        if neg_id not in self.train_items[u] and neg_id not in neg_batch:
          neg_batch.append(neg_id)
      return neg_batch
    

    pos_items, neg_items = [], []
    for u in users:
      pos_items += sample_pos_items_for_u(u, 1)
      neg_items += sample_neg_items_for_u(u, 1)

    return users, pos_items, neg_items


In [0]:
# small test to verify whether we can load data
dataReader = DataReader(data_path, 10)
dataReader.sample()

([52204, 38750, 22561, 40757, 36173, 24461, 32265, 48823, 23598, 8338],
 [58144, 66518, 65558, 77227, 54514, 17201, 8207, 89156, 8110, 10353],
 [9709, 31828, 76941, 58085, 60899, 89691, 64700, 80384, 46681, 27799])

#The steps to take
As a rough outline, we can sketch the steps to be taken to train a Neural Graph Collaborative Filtering system as follows.

1. Implement the NGCF model
  1. Create a user-item interaction graph from our data.
  1. Create a NGCF Layer that can be used to perform a MessagePassing step.
1. Train all the parameters
  1. Construct the matrices W1_l, W2_l and E_l for each step l, initialize them with the Xavier Initializer.
  1. Sample a batch from the training data. Apply L steps of message passing, and save all the matrices W1_{1..L}, W2_{1..L}, E_{1..L}, repeat until all training data are sampled once.
  1. Construct the final embeddings for each user and item, by concatenating the embeddings for a user or item from each step, so **e_u** = **e_u^1** || ... || **e_u^L**
  1. Using the final embeddings, compute the prediction for each (user, item) pair, and compute the BPR loss.
  1. Use the BPR loss to update the matrices W1 and W2, repeat until <amount> epochs are done.
1. Generate the test output, compute recall and ndgc, compare them to the Table 3
  1. Use the trained matrices W1 and W2 to find the prediction values for each (user, item) pair in the test data.
  1. Compute the recall and normalized discounted cumulative gain (ndgc)

Since we have to compare results for step l, and the final embeddings are calculated by concatenation over the steps, we can simply perform all L steps (4 in this case), and then do the analysis over subsets of the W1 and W2 matrices.

Hyperparameters to consider:
 - Negative slope of NGCFLayer#LeakyReLU
 - ...


# step 1
The initial embedding is already performed in the data sets and thus we can consider to already have our embedding table. In the paper's work, it is explained that in their NGCF framework the embeddings are refined by propagating them on the user-item interaction graph. 

This implies that we need to construct an interaction graph, and update its weights according to the message construction/message aggregation functions.

By using PyTorch and PyTorch Geometric (PyG) we are able to construct a graph neural network and perform various operations. We are planning to use this framework. 

## GCN with torch_geometric (PyG)
Some of its steps are described in this blog post:
https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8

There is also a google colab which trains a GCN to identify 'spammers'
https://colab.research.google.com/github/zaidalyafeai/Notebooks/blob/master/Deep_GCN_Spam.ipynb#scrollTo=_4_eVOI2M4Uo

**Importing torch_geometric 1.3.2**

`torch_geometric` is a geometric deep learning extension library for PyTorch. We don't use the latest version (loaded by default by google colab) due to inconsistencies with PyTorch. This also means we have to downgrade a few other packages aswell.

In [0]:
pip install torch===1.2.0 torchvision===0.4.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 748.9MB 20kB/s 
     |████████████████████████████████| 8.8MB 39.4MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0


In [0]:
import torch
# we use torch version 1.2.0 instead of the latest due to dependency errors
print(torch.__version__)

1.2.0


In [0]:
# these were the corresponding versions for torch-geometric 1.3.2, released 4 oct 2019, which runs on torch 1.2.0.
# grab some coffee, this might take a while
!pip install torch-scatter==1.3.1
!pip install torch-sparse==0.4.0
!pip install torch-cluster==1.4.4
!pip install torch-spline-conv==1.1.0
!pip install torch-geometric==1.3.2

  Created wheel for torch-scatter: filename=torch_scatter-1.3.1-cp36-cp36m-linux_x86_64.whl size=2724892 sha256=fe5b6815e6a9ea56f7d253b56eaf1915335b3e9d196333bafd9a0b3921bac914
  Stored in directory: /root/.cache/pip/wheels/7f/21/0b/c42fa9353ceec5e87464599e470a03e4250ec667b4a392fa7d
Successfully built torch-scatter
  Created wheel for torch-sparse: filename=torch_sparse-0.4.0-cp36-cp36m-linux_x86_64.whl size=3552064 sha256=db2d00df64416105387efc5c5d96d7169f62b807a6bc1c5ed5e69daffde0d95a
  Stored in directory: /root/.cache/pip/wheels/9d/83/0a/38ea460df5586a075b877fe089619e5238487712a0645940bd
Successfully built torch-sparse
  Created wheel for torch-cluster: filename=torch_cluster-1.4.4-cp36-cp36m-linux_x86_64.whl size=14436689 sha256=57611e49a99599f867acb9ac47f30bc8179b888a666821d0dce0e592f09e0265
  Stored in directory: /root/.cache/pip/wheels/20/7b/ab/b3e266920055d1e51988f93a99ef8df62e399b234c8d50527f
Successfully built torch-cluster
  Created wheel for torch-spline-conv: filename=tor

In [0]:
import torch
import torch_geometric


In [0]:
# verify that torch geometric is imported, should be 1.3.2
print(torch_geometric.__version__)

1.3.2


**Creating the interaction graph**

First we create an interaction graph. The Data object from torch_geometric represents a graph structure, and as such, we should create one given our data.

In [0]:
from torch_geometric.data import Data

# get data
dataReader = DataReader(batch_size=100)
edges = []
# define an edge for every user and item
for user in range(dataReader.n_users):
  pos_items = dataReader.train_items[user]
  for item in pos_items:
    # increase the item id by the number of users to distinct the nodes from users
    edges.append([user, item + dataReader.n_users])
    edges.append([item + dataReader.n_users, user])

# following example from https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# we can extend x with any number of features we want.
# for now, we only have it include its corresponding userid or itemid, but we might want to extend this
x = torch.tensor(list(range(dataReader.n_users)) + list(range(dataReader.n_items)), dtype=torch.long)

# data is a representation of the directed graph
data = Data(x=x, edge_index=edge_index)



Data is a representation of our interaction graph

In [0]:
# torch_geometric.data.Data provides a number of utility functions
# these prints are a check to verify if our graph seems valid
print(data.is_undirected())
print(data.is_directed())
print(data.num_edges)
print(data.num_nodes)
print(data.contains_isolated_nodes())
print(data.contains_self_loops())

True
False
4761412
144240
False
False


# step 2

First we initialize the initial weights and embeddings. This is performed by [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf).

In [0]:
import math

In [0]:
def xavier(m, h):
  return torch.Tensor(m, h).uniform_(-1, 1)*math.sqrt(6./(m+h))

class NGCF(object):
  def __init__(self, n_users, n_items, emb_dim=64, n_layers=4):
    self.n_users = n_users
    self.n_items = n_items
    self.emb_dim = emb_dim
    self.weight_size = list(np.repeat(emb_dim, n_layers))
    self.n_layers = n_layers

  def _init_weights(self):
    all_weights = dict()
    all_weights['user_embedding'] = xavier(self.n_users, self.emb_dim) 
    all_weights['item_embedding'] = xavier(self.n_items, self.emb_dim)
    self.weight_size_list = [self.emb_dim] + self.weight_size
    for k in range(self.n_layers):
      all_weights['W_gc_%d' % k] = xavier(self.weight_size_list[k], self.weight_size_list[k+1])
      all_weights['b_gc_%d' % k] = xavier(1, self.weight_size_list[k+1])
      all_weights['W_bi_%d' % k] = xavier(self.weight_size_list[k], self.weight_size_list[k + 1])
      all_weights['b_bi_%d' % k] = xavier(1, self.weight_size_list[k + 1])
      all_weights['W_mlp_%d' % k] = xavier(self.weight_size_list[k], self.weight_size_list[k+1])
      all_weights['b_mlp_%d' % k] = xavier(1, self.weight_size_list[k+1])
    return all_weights



In [0]:
# set initial weight
# emd_dim and n_layers can be tweaked to our liking
ncfg_obj = NGCF(n_users=dataReader.n_users, n_items=dataReader.n_items, emb_dim=64, n_layers=4)

# store all our initial weights in some dict object
ncfg_weights = ncfg_obj._init_weights()

# optional prints: check the amount of user embeddings as a verification
print(ncfg_weights.keys())
print(len(ncfg_weights['user_embedding']))

dict_keys(['user_embedding', 'item_embedding', 'W_gc_0', 'b_gc_0', 'W_bi_0', 'b_bi_0', 'W_mlp_0', 'b_mlp_0', 'W_gc_1', 'b_gc_1', 'W_bi_1', 'b_bi_1', 'W_mlp_1', 'b_mlp_1', 'W_gc_2', 'b_gc_2', 'W_bi_2', 'b_bi_2', 'W_mlp_2', 'b_mlp_2', 'W_gc_3', 'b_gc_3', 'W_bi_3', 'b_bi_3', 'W_mlp_3', 'b_mlp_3'])
52642


Message passing, constructing and aggregation aswell as training neural networks is all included in the `torch_geometric` package.


https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html


For each user-item pair (u,i), we define the message from i to u as

$m_{u \leftarrow i} = \dfrac{1}{\sqrt{|N_u||N_i|}} (W_1 e_i + W_2(e_i \odot e_u))$

Where $N_u, N_i$ are the first hop neighbors of $u$ and $i$, $W_1, W_2$ are trainable weight matrices to distill useful information for propagation, and $e_i$ and $e_u$ are embeddings of the users and items.

The embedding for a user (or item) is updated through message aggregation as 

$e_u^{(l)} = LeakyReLU(m_{u \leftarrow u} + \sum_{i \in N_u} m_{u \leftarrow i})$

If the graph is constructed properly, this means that this simply a summation over all edges (when selfloops are included), followed by a LeakyReLU activiation.

In [0]:
from torch.nn import LeakyReLU, Linear, MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class NGCFLayer(MessagePassing):
  def __init__(self, in_channels, out_channels):
    super(NGCFLayer, self).__init__(aggr='add') # Summation as aggregation
    # The W1 matrix
    self.lin1 = Linear(bias=false)
    # The W2 matrix
    self.lin2 = Linear(bias=false)
    self.act = LeakyReLU()

  def forward(self, x, edge_index):
    # Add self loops
    edge_index, _ = remove_self_loops(edge_index)
    edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

    # Compute normalization
    row, col = edge_index
    deg = degree(row, x.size(0), dtype=x.dtype)
    deg_inv_dqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)
  
  def message(self, x_i, x_j, norm):
    # Construct message
    message = self.lin1(x_j)
    # Only add the second term if it is not a self loop
    if x_i.data_ptr() != x_j.data_ptr():
      message += self.lin2(torch.mul(x_i, x_j))
    return message

  def update(self, aggr_out):
    # Return the LeakyReLU result
    return self.act(aggr_out)